In [18]:
import os
from pathlib import Path
from shutil import copyfile

ROOT = Path("msunique/Data")
DATA_DIR = Path("data")

In [21]:
for dir, _, files in os.walk(ROOT):
    dir = Path(dir)
    for file in files:
        if file.endswith('.json'):
            path = dir / file
            dest = DATA_DIR / f"{dir.stem}_{file}"
            print(f"Copying {path} to {dest}")
            copyfile(path, dest)


Copying msunique/Data/Raiffeisen/2022.json to data/Raiffeisen_2022.json
Copying msunique/Data/Raiffeisen/2023.json to data/Raiffeisen_2023.json
Copying msunique/Data/Raiffeisen/2021.json to data/Raiffeisen_2021.json
Copying msunique/Data/ABB/2022.json to data/ABB_2022.json
Copying msunique/Data/ABB/2023.json to data/ABB_2023.json
Copying msunique/Data/ABB/2021.json to data/ABB_2021.json
Copying msunique/Data/Siemens/2022.json to data/Siemens_2022.json
Copying msunique/Data/Siemens/2023.json to data/Siemens_2023.json
Copying msunique/Data/Siemens/2021.json to data/Siemens_2021.json
Copying msunique/Data/IBM/2022.json to data/IBM_2022.json
Copying msunique/Data/IBM/2023.json to data/IBM_2023.json
Copying msunique/Data/IBM/2021.json to data/IBM_2021.json
Copying msunique/Data/PostFinance/2022.json to data/PostFinance_2022.json
Copying msunique/Data/PostFinance/2023.json to data/PostFinance_2023.json
Copying msunique/Data/PostFinance/2021.json to data/PostFinance_2021.json
